In [4]:
%pip install nltk
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
%pip install networkx
import networkx as nx


   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---- ----------------------------------- 0.2/1.7 MB 3.1 MB/s eta 0:00:01
   ----------------- ---------------------- 0.7/1.7 MB 6.6 MB/s eta 0:00:01
   --------------------- ------------------ 0.9/1.7 MB 5.7 MB/s eta 0:00:01
   ------------------------- -------------- 1.1/1.7 MB 5.2 MB/s eta 0:00:01
   --------------------------- ------------ 1.2/1.7 MB 5.3 MB/s eta 0:00:01
   --------------------------- ------------ 1.2/1.7 MB 5.3 MB/s eta 0:00:01
   --------------------------- ------------ 1.2/1.7 MB 5.3 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 4.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\AKSHAY\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\AKSHAY\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [25]:
def read_article(text):
    sentences = sent_tokenize(text)
    return sentences

def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []

    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]

    all_words = list(set(sent1 + sent2))

    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)

    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1

    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1

    return 1 - cosine_distance(vector1, vector2)

def build_similarity_matrix(sentences, stop_words):
    similarity_matrix = np.zeros((len(sentences), len(sentences)))

    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2:
                continue
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)
    return similarity_matrix

def generate_summary(text, threshold=0.2):
    stop_words = stopwords.words('english')
    summarize_text = []

    sentences = read_article(text)
    sentence_similarity_matrix = build_similarity_matrix(sentences, stop_words)
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)
    scores = nx.pagerank(sentence_similarity_graph)

    # Sort the sentences by their PageRank scores
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

    # Include sentences in the summary if their score is above the threshold
    for score, sentence in ranked_sentences:
        if score > threshold:
            summarize_text.append(sentence)

    # Combine selected sentences to create the final summary
    summary = ' '.join(summarize_text)
    return summary

# Example usage:
text = "Dave watched as the forest burned up on the hill, only a few miles from her house. The car had been hastily packed and Marta was inside trying to round up the last of the pets. Dave went through his mental list of the most important papers and documents that they couldn't leave behind. He scolded himself for not having prepared these better in advance and hoped that he had remembered everything that was needed. He continued to wait for Marta to appear with the pets, but she still was nowhere to be seen."
summary = generate_summary(text)
print(summary)


He continued to wait for Marta to appear with the pets, but she still was nowhere to be seen. Dave watched as the forest burned up on the hill, only a few miles from her house. Dave went through his mental list of the most important papers and documents that they couldn't leave behind. The car had been hastily packed and Marta was inside trying to round up the last of the pets. He scolded himself for not having prepared these better in advance and hoped that he had remembered everything that was needed.
